In [1]:
from datetime import datetime, date,timedelta
import psycopg2
import re
import pandas as pd
import numpy as np
import warnings
import io
import time
import ipywidgets as widgets
from IPython.display import display
#from google.colab import files

warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)
warnings.simplefilter(action='ignore', category=UserWarning)

In [2]:
## make a Connection

In [3]:
con1=psycopg2.connect(
        dbname= '', 
        host='', 
        port= '',
        user= '', 
        password= '')
cur1 = con1.cursor()

In [4]:
##Loading the Keyword file
model = pd.read_excel('model.xlsx',sheet_name="keywords")
model.columns = [x for x in model.iloc[0]]
model = model[1:]
print(model.shape)
model.head()

(199, 7)


,Keywords,Identifier,Main_Subset,Category,Transformations,Superset,Subset
1,Warm Oil,Sub category,Subset,Remedies,"_hot_%_oil_, warm%_oil_, _garam tel_, _garam oil_",Home Remedies,Warm Oil
2,Avocado paste,Sub category,Subset,Remedies,avocado%paste,Home Remedies,Avocado paste
3,Coconut oil,Sub category,Subset,Remedies,coconut oil,Home Remedies,Coconut oil
4,Yogurt,Sub category,Subset,Remedies,"_yogurt_, _dahi_, _curd_",Home Remedies,Yogurt
5,Massage,Sub category,Subset,Remedies,"massage, maalish_, malish_, maalis_, malis_",Home Remedies,Massage


In [5]:
##Creating a transformation list and cleaning it

transformation = model.loc[model['Main_Subset']=='Main','Transformations']
transformation = list(transformation)
transformation = [ x.split(',') for x in transformation]
transformation = [y for x in transformation for y in x]
transformation = [x.lower() for x in transformation ]
transformation = [x.lstrip() for x in transformation ]
transformation = [x.replace("_",' ') if (len(x)>6) else x for x in transformation]
transformation = [x.replace("_",' ') for x in transformation]
#transformation.remove('honest')
#transformation

##Creating text lower like %word% condition to feed into query

text_matching_condition = []
for x in transformation:
    text_matching_condition.append(f" or textlower ilike '%{x}%'")

#Removing the 'or' from first line    
text_matching_condition[0] = text_matching_condition[0].replace(" or ",'')
text_matching_condition = ','.join(text_matching_condition)
text_matching_condition = text_matching_condition.replace(",",'')
text_matching_condition

#Creating groupids into tuples to feed into query

"textlower ilike '%shampo%' or textlower ilike '% baal %dhulna%' or textlower ilike '% baal %dhona%' or textlower ilike '%baalo%dhulna%' or textlower ilike '%baalo%dhona%' or textlower ilike '% hair%serum%' or textlower ilike '% hair% oil %' or textlower ilike '% oil % hair%' or textlower ilike '% baal % tel %' or textlower ilike '%baalo% tel %' or textlower ilike '% hair% mask%' or textlower ilike '% hair% masque%' or textlower ilike '%dandruf%' or textlower ilike '% roosi %' or textlower ilike '% rusi %' or textlower ilike '% hair%' or textlower ilike '% baal %' or textlower ilike '%baalo %' or textlower ilike '%scalp%' or textlower ilike '%khopdi%' or textlower ilike '%hair & care%' or textlower ilike '%hair and care%' or textlower ilike '%hair n care%' or textlower ilike '%bblunt%' or textlower ilike '%b blunt%' or textlower ilike '%b-blunt%' or textlower ilike '%wella%color%' or textlower ilike '%color%wella%' or textlower ilike '%naturigin%color%' or textlower ilike '%color%natur

In [7]:
group_data = pd.read_excel("model.xlsx",sheet_name='bajaj')
groupids = group_data['groupid']
groupids = [x for x in groupids if str(x)!= 'nan' ]
groupids = tuple(groupids)
print(len(groupids))
# groupids

47


In [8]:
# Function to get transformations for a specific keyword from the model
def generateExcludeWordsSQL(words_string, case=False):
    
    il_var = 'not ilike' if case is False else 'not like'
    tformations = words_string.split(',')
    tformations = ["'%" + t.strip().lower().replace('_',' ') + "%'" for t in tformations]
    tformations = f'textlower {il_var} ' + f' and textlower {il_var} '.join(tformations)
    return 'and (' + tformations + ')'

In [9]:
exclude_text = generateExcludeWordsSQL('baby,babies,infant,_kid_,_kids_,baccha,child,shishu,bachha,toddler')
# exclude_text = '' # uncomment and exec to clear exclude_text
exclude_text

"and (textlower not ilike '%baby%' and textlower not ilike '%babies%' and textlower not ilike '%infant%' and textlower not ilike '% kid %' and textlower not ilike '% kids %' and textlower not ilike '%baccha%' and textlower not ilike '%child%' and textlower not ilike '%shishu%' and textlower not ilike '%bachha%' and textlower not ilike '%toddler%')"

In [10]:
start_date = widgets.DatePicker(description='Start Date', disabled=False)
end_date = widgets.DatePicker(description='End Date', disabled=False)
display(start_date, end_date)

DatePicker(value=None, description='Start Date')

DatePicker(value=None, description='End Date')

In [11]:
date_range = f"createdatutc >= '{start_date.value.strftime('%Y-%m-%d')}' and createdatutc <= '{end_date.value.strftime('%Y-%m-%d')} 23:59:59'"
date_range

"createdatutc >= '2021-02-01' and createdatutc <= '2022-04-30 23:59:59'"

In [12]:
x1 = f"""
Select createdatutc,groupid,groupname,textlower,sentimentrvalue,type
from 
bd_cs_prod_conversations 
where 
groupid in {groupids} 
and ({text_matching_condition})
{exclude_text}
and ({date_range})
union
Select createdatutc,groupid,groupname,textlower,sentimentrvalue,type
from bd_cs_prod_conversations 
where parentsourceid IN (
Select sourceid
from 
bd_cs_prod_conversations 
where 
groupid in {groupids} 
and ({text_matching_condition})
{exclude_text}
and (type = 'Post') 
and (grouptype = 'Facebook') 
and ({date_range})
)
and (type = 'Comment')
and ({date_range})
{exclude_text}
"""

In [13]:
%%time
cur1.execute(x1)
w = cur1.fetchall()
y = pd.DataFrame(w)
y.columns = ['createdatutc','groupid','groupname','textlower','senti','type']

CPU times: total: 6.3 s
Wall time: 31min 24s


In [14]:
y.to_pickle('new-data.pickle')

In [15]:
y.textlower = y.textlower.replace('',np.NaN)
y=y.dropna(axis=0, subset='textlower')
catconv_0 = y.shape[0]
print('Category conversations:',catconv_0)

Category conversations: 247803


In [16]:
y[y.textlower.str.contains('baby',case=False)].type.value_counts()

Series([], Name: type, dtype: int64)

In [20]:
y.to_pickle('bajaj_data.pickle')

In [ ]:
#make a copy of dataframe
w=y.copy()

In [5]:
y = pd.read_pickle('bajaj_data.pickle')
y.shape

(452885, 6)

In [62]:
y_with_class = group_data[['groupClass','groupid']].merge(y, 'left', on='groupid')
y_with_class.shape

(452885, 7)

In [63]:
y_with_class.to_pickle('data_with_class.pickle')

In [7]:
y = pd.read_pickle('data_with_class.pickle')
y.shape

(452885, 7)

In [18]:
y.head()

,createdatutc,groupid,groupname,textlower,senti,type
0,2021-02-04 23:49:17,dfbd4a0d-cf2c-4398-a882-c3b5f9f2711a,F2FGrooming - Haircare•SkinCare•Modern Etiquet...,olive or almond oil massage into cuticles and ...,positive,Comment
1,2021-02-05 01:56:35,ee4e4185-7b93-4082-b5ec-46ff2a36dc01,"Make Up, Skin & Hair Care Tips",nice\nyes hairs clean ni honge to uska effect ...,positive,Comment
2,2021-02-05 05:02:26,ee4e4185-7b93-4082-b5ec-46ff2a36dc01,"Make Up, Skin & Hair Care Tips",anjali ma'am which shampoo n conditioner i hav...,neutral,Comment
3,2021-02-05 05:49:27,1052cb49-5c0c-44c7-b4dd-76a2c23c0e9a,"Makeup, Beauty, Hair Fall and Skin",certified in makeup in ahmedabad & vadodara\n ...,positive,Post
4,2021-02-05 05:40:35,ee4e4185-7b93-4082-b5ec-46ff2a36dc01,"Make Up, Skin & Hair Care Tips",how often do you colour your hair ?,neutral,Post


In [29]:
# y_with_class.to_excel('bajaj_data.xlsx', index=False)

In [19]:
y.createdatutc = y.createdatutc.dt.strftime('%B-%Y')
y.createdatutc = pd.to_datetime(y.createdatutc)

In [22]:
#store count of conversation on month basic in 'y1' data frame
y1 = y.copy()
y1 = y1.groupby(['createdatutc'])[['textlower']].count().T
y1.columns = y1.columns.strftime('%b-%y')
y1 = y1.reset_index().rename_axis(None, axis=1)
y1.to_csv('bajaj-catconv_monthwise_new.csv', index=False)
y1.rename(columns={'index':'Count'}, inplace=True)
y1

,Count,Feb-21,Mar-21,Apr-21,May-21,Jun-21,Jul-21,Aug-21,Sep-21,Oct-21,Nov-21,Dec-21,Jan-22,Feb-22,Mar-22,Apr-22
0,textlower,16486,22715,19712,16767,21603,22020,17245,15381,23048,17071,13242,16427,9354,7094,9638


In [ ]:
##download conversation dump
y.to_csv("s26mentions.csv")
files.download("s26mentions.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
##Creating a dictionary of keys as keywords and it's values as transformations

In [23]:
d = {}

for i in range(model.shape[0]):
    key = model.Keywords.iloc[i]
    trans = [x.strip() for x in model.Transformations.iloc[i].split(',')]
    if key in d.keys():
        d[key].extend(trans)
    else:
        d[key] = trans
    # print(f'{key}: {d[key]}')
    d[key] = list(set(d[key]))
    
for key, tf in d.items():
    d[key] = '|'.join(tf).lower().replace('%','.*').replace('_',' ')
    
# d

In [24]:
def getMentions(text, value):
    if re.search(value, text.lower(), flags=re.DOTALL): return 1
    else: return 0

In [25]:
tick = time.time()
for key,value in d.items():
    y[key] = y.textlower.apply(lambda x: getMentions(str(x), value))
time.time()-tick

279.4600718021393

In [26]:
y.shape

(247803, 205)

In [38]:
y.to_pickle('bajaj-convdump1_0_new.pickle')

In [26]:
#y['#'] = y['#'].replace({1: 'Campaign'})
#y['#'] = y['#'].replace({0: 'Non Campaign'})

In [12]:
# y = pd.read_pickle('toddler-convdump1_0.pickle')
# y.shape

(884842, 191)

In [29]:
hashtag = y.textlower.str.contains('#')
hashtag.shape

(247803,)

In [30]:
hashtag.value_counts()

False    224959
True      22844
Name: textlower, dtype: int64

In [31]:
hashtag1_0 = hashtag.apply(lambda x: 1 if x is True else 0)

In [36]:
y.insert(6, '#', hashtag1_0)

In [37]:
y.head()

,createdatutc,groupid,groupname,textlower,senti,type,#,Warm Oil,Avocado paste,Coconut oil,...,Herbal essence,Himalaya,Khadi,Wella,Plum Shampoo,Body Shop Shampoo,Purplle,Amazon,Flipkart,Nykaa
0,2021-02-01,dfbd4a0d-cf2c-4398-a882-c3b5f9f2711a,F2FGrooming - Haircare•SkinCare•Modern Etiquet...,olive or almond oil massage into cuticles and ...,positive,Comment,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2021-02-01,ee4e4185-7b93-4082-b5ec-46ff2a36dc01,"Make Up, Skin & Hair Care Tips",nice\nyes hairs clean ni honge to uska effect ...,positive,Comment,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2021-02-01,ee4e4185-7b93-4082-b5ec-46ff2a36dc01,"Make Up, Skin & Hair Care Tips",anjali ma'am which shampoo n conditioner i hav...,neutral,Comment,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2021-02-01,1052cb49-5c0c-44c7-b4dd-76a2c23c0e9a,"Makeup, Beauty, Hair Fall and Skin",certified in makeup in ahmedabad & vadodara\n ...,positive,Post,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2021-02-01,ee4e4185-7b93-4082-b5ec-46ff2a36dc01,"Make Up, Skin & Hair Care Tips",how often do you colour your hair ?,neutral,Post,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [45]:
y = pd.concat([y.iloc[:,:6], y.select_dtypes(include='int64').astype('int8')], axis=1)
y.shape

(247803, 206)

In [48]:
#all comments containing 1,0 for all keywords
y.to_csv('bajaj-conversations_new.csv',encoding="utf-8-sig",index = False)

In [ ]:
##Getting values of keywords and all it's count

In [23]:
y.to_pickle('bajaj-convdump1_0.pickle')

In [2]:
y = pd.read_pickle('bajaj-convdump1_0.pickle')
y.shape

(452885, 174)

In [18]:
#store count of conversation on month basic in 'y1' data frame
y1 = y.copy()
y1 = y1.groupby(['createdatutc','groupClass','#']).textlower.count().unstack(level=0)
y1.columns = y1.columns.strftime('%b-%y')
y1 = y1.reset_index().rename_axis(None, axis=1)
y1.to_csv('bajaj-catconv_monthwise.csv', index=False)
y1.insert(1, 'ConvType', y1['#'].apply(lambda x: 'Campaign' if x == 1 else 'Non-Campaign'))
y1.drop(columns='#',inplace=True)
y1

,groupClass,ConvType,Feb-21,Mar-21,Apr-21,May-21,Jun-21,Jul-21,Aug-21,Sep-21,Oct-21,Nov-21,Dec-21,Jan-22,Feb-22,Mar-22,Apr-22
0,Bajaj,Non-Campaign,15864,21247,18517,15637,18684,19266,15611,14701,21083,14436,11756,14962,7724,6515,8956
1,Bajaj,Campaign,622,1468,1195,1130,2919,2754,1634,680,1965,2635,1486,1465,1630,579,682
2,Non-Bajaj,Non-Campaign,16848,14987,10035,10483,13135,17595,12787,15340,19089,15473,11597,10075,8252,10816,11310
3,Non-Bajaj,Campaign,487,700,435,343,426,423,342,484,792,609,634,431,354,467,333


In [49]:
# campaign = y[(y['#'] == 1) & (y.groupClass == 'Bajaj')]
campaign = y[y['#'] == 1]

In [51]:
campaign = y.copy()
campaign

,createdatutc,groupid,groupname,textlower,senti,type,#,Warm Oil,Avocado paste,Coconut oil,...,Herbal essence,Himalaya,Khadi,Wella,Plum Shampoo,Body Shop Shampoo,Purplle,Amazon,Flipkart,Nykaa
0,2021-02-01,dfbd4a0d-cf2c-4398-a882-c3b5f9f2711a,F2FGrooming - Haircare•SkinCare•Modern Etiquet...,olive or almond oil massage into cuticles and ...,positive,Comment,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2021-02-01,ee4e4185-7b93-4082-b5ec-46ff2a36dc01,"Make Up, Skin & Hair Care Tips",nice\nyes hairs clean ni honge to uska effect ...,positive,Comment,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2021-02-01,ee4e4185-7b93-4082-b5ec-46ff2a36dc01,"Make Up, Skin & Hair Care Tips",anjali ma'am which shampoo n conditioner i hav...,neutral,Comment,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2021-02-01,1052cb49-5c0c-44c7-b4dd-76a2c23c0e9a,"Makeup, Beauty, Hair Fall and Skin",certified in makeup in ahmedabad & vadodara\n ...,positive,Post,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2021-02-01,ee4e4185-7b93-4082-b5ec-46ff2a36dc01,"Make Up, Skin & Hair Care Tips",how often do you colour your hair ?,neutral,Post,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
261451,2022-03-01,1e4f28b0-73d5-4389-ba8b-7c25635daad0,"Mom's lifestyle, beauty and health tips",Superb,positive,Comment,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
261452,2022-03-01,1e4f28b0-73d5-4389-ba8b-7c25635daad0,"Mom's lifestyle, beauty and health tips",Awsome,positive,Comment,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
261453,2022-03-01,1e4f28b0-73d5-4389-ba8b-7c25635daad0,"Mom's lifestyle, beauty and health tips",Awesome 👍,positive,Comment,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
261454,2022-03-01,1e4f28b0-73d5-4389-ba8b-7c25635daad0,"Mom's lifestyle, beauty and health tips",Harshala Ramteke thank u dear 🙏,positive,Comment,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [34]:
# non_campaign = y[(y['#'] == 0) & (y.groupClass == 'Bajaj')]
non_campaign = y[y['#'] == 0]
non_campaign

,groupClass,groupid,createdatutc,groupname,textlower,senti,type,#,Warm Oil,Avocado paste,...,Dabur Vatika,Emami,Herbal essence,Himalaya,Khadi,Wella,Purplle,Amazon,Flipkart,Nykaa
0,Bajaj,9836d720-5665-46d4-bebe-ae8866bc6307,2021-04-01,health and beauty tips,"premium alovera gel jar \niso, gmp, natural ce...",positive,Post,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Bajaj,9836d720-5665-46d4-bebe-ae8866bc6307,2021-02-01,health and beauty tips,some homemade remedies for your hair and skin,neutral,Post,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Bajaj,9836d720-5665-46d4-bebe-ae8866bc6307,2021-03-01,health and beauty tips,you can add these ingredients into your shampoo,neutral,Post,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Bajaj,9836d720-5665-46d4-bebe-ae8866bc6307,2021-03-01,health and beauty tips,some simple hair care tips,positive,Post,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Bajaj,9836d720-5665-46d4-bebe-ae8866bc6307,2021-03-01,health and beauty tips,some home remedies for dandruff !,negative,Post,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
452880,Non-Bajaj,7920405f-a8e4-4a4d-b69f-c1fc85b4d601,2022-03-01,Arpita's & Her Team সাজকথা( female group),soma ghosh 280:/\n6pc,neutral,Comment,0,0,0,...,0,0,0,0,0,0,0,0,0,0
452881,Non-Bajaj,7920405f-a8e4-4a4d-b69f-c1fc85b4d601,2022-04-01,Arpita's & Her Team সাজকথা( female group),pp,neutral,Comment,0,0,0,...,0,0,0,0,0,0,0,0,0,0
452882,Non-Bajaj,7920405f-a8e4-4a4d-b69f-c1fc85b4d601,2022-04-01,Arpita's & Her Team সাজকথা( female group),offferrpp,neutral,Comment,0,0,0,...,0,0,0,0,0,0,0,0,0,0
452883,Non-Bajaj,7920405f-a8e4-4a4d-b69f-c1fc85b4d601,2022-04-01,Arpita's & Her Team সাজকথা( female group),pp,neutral,Comment,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [234]:
for col in campaign.columns[7:]:
    print(col,campaign[col].sum())

# 7260
Warm Oil 22
Avocado paste 0
Coconut oil 332
Yogurt 114
Massage 121
Supplements 10
Drink Water 16
Hair Spa 158
Salon 489
Amla Juice 1
Egg Mask 0
Consult Specilaist 2
Balanced Diet 128
Onion Juice 62
Omega 3 5
Shampoo 1219
Conditioner 230
Hair Color 476
Hair Dye 30
Hair Dryer 51
Hair Straightener 103
Hair Brush 38
Comb 57
Hair Spray 129
Hair Serum 91
Hair Mask 199
Hair Oil 850
Hair Curler 7
Gel 241
Root Touch Up Spray 2
Trimmers & Shavers 3
Grooming Kits 0
After-shave lotion 0
Hair fall 1422
Dandruff 615
Split Ends 45
Grey Hair 111
Lice 8
Bald 4
Frizzy Hair 168
Dry Hair 64
Itching 2
Hair Thinning 7
Hair Breakage 9
Dull Hair 20
Styling 647
Moisturize 103
Natural 754
Strengthen 66
Price 47
Paraffin free 0
Fragrance 120
Non Sticky 6
Chemical free 205
Hair Growth 482
Nourishment 126
Cures Dandruff 112
Stops Hairfall 293
Soft & Smooth Hair 100
Detangle 5
Stops Itch 9
Treats Lice 1
Grey hair treatment 5
Argan 60
Vinegar 62
Onion 443
Almond 47
Coconut 399
Amla 29
Castor 32
Sesame 4
Coffe

In [ ]:
##Creating a dataframe with monthwise count of every word

In [25]:
# campaign['createdatutc'] = campaign['createdatutc'].dt.strftime('%B-%Y')

In [54]:
campaign2 = campaign.copy()
campaign2 = campaign2.groupby(['createdatutc'])[list(campaign2.columns[7:])].sum().T
# campaign2 = campaign2.stack(level=0,).reset_index().set_index(['createdatutc']).unstack(level=0)
# campaign2 = campaign2.droplevel(level=0, axis=1)
campaign2.columns = campaign2.columns.strftime('%b-%y')
campaign2 = campaign2.reset_index().rename_axis(None, axis=1)
campaign2.rename(columns={'index':'Keywords'}, inplace=True)
# campaign2.insert(0, 'Communities', 'Overall')
campaign2.to_csv('bajaj-keyword_mentions_new.csv', index=False)
campaign2

,Keywords,Feb-21,Mar-21,Apr-21,May-21,Jun-21,Jul-21,Aug-21,Sep-21,Oct-21,Nov-21,Dec-21,Jan-22,Feb-22,Mar-22,Apr-22
0,Warm Oil,13.0,16.0,8.0,15.0,14.0,35.0,25.0,15.0,25.0,21.0,9.0,19.0,10.0,9.0,8.0
1,Avocado paste,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Coconut oil,234.0,215.0,202.0,234.0,213.0,311.0,220.0,169.0,213.0,177.0,124.0,134.0,98.0,107.0,165.0
3,Yogurt,85.0,93.0,80.0,86.0,125.0,197.0,94.0,72.0,182.0,118.0,57.0,49.0,41.0,47.0,84.0
4,Massage,101.0,90.0,103.0,92.0,86.0,188.0,142.0,116.0,155.0,134.0,69.0,80.0,41.0,100.0,71.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
194,Body Shop Shampoo,0.0,3.0,2.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,2.0,0.0,0.0,0.0
195,Purplle,11.0,17.0,7.0,10.0,14.0,9.0,9.0,14.0,10.0,10.0,4.0,7.0,8.0,40.0,5.0
196,Amazon,64.0,47.0,24.0,54.0,70.0,41.0,24.0,67.0,34.0,88.0,62.0,60.0,34.0,24.0,52.0
197,Flipkart,3.0,9.0,7.0,31.0,25.0,7.0,11.0,20.0,14.0,25.0,19.0,19.0,15.0,10.0,32.0


In [ ]:
for col in non_campaign.columns[7:]:
    print(col,non_campaign[col].sum())

In [37]:
non_campaign2 = non_campaign.copy()
non_campaign2 = non_campaign2.groupby(['createdatutc'])[list(non_campaign2.columns[7:])].sum().T
# non_campaign2 = non_campaign2.stack(level=0,).reset_index().set_index(['createdatutc']).unstack(level=0)
# non_campaign2 = non_campaign2.droplevel(level=0, axis=1)
non_campaign2.columns = non_campaign2.columns.strftime('%b-%y')
non_campaign2 = non_campaign2.reset_index().rename_axis(None, axis=1)
non_campaign2.rename(columns={'index':'Keywords'}, inplace=True)
non_campaign2.insert(0, 'Communities', 'Overall')
non_campaign2

,Communities,Keywords,Feb-21,Mar-21,Apr-21,May-21,Jun-21,Jul-21,Aug-21,Sep-21,Oct-21,Nov-21,Dec-21,Jan-22,Feb-22,Mar-22,Apr-22
0,Overall,#,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Overall,Warm Oil,17,13,14,26,19,35,17,15,15,15,21,22,18,11,10
2,Overall,Avocado paste,0,0,0,0,0,0,3,0,0,0,0,0,0,0,1
3,Overall,Coconut oil,459,423,350,454,462,527,312,275,240,254,218,210,182,175,235
4,Overall,Yogurt,114,113,106,118,175,185,93,87,86,63,67,44,60,71,99
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
162,Overall,Wella,3,12,6,15,14,18,10,10,6,7,15,33,5,2,3
163,Overall,Purplle,22,21,10,16,18,15,7,17,14,16,9,10,10,17,11
164,Overall,Amazon,106,87,40,99,82,75,44,57,45,86,61,53,45,46,36
165,Overall,Flipkart,6,14,7,34,12,6,10,11,16,14,7,12,14,12,5


In [ ]:
##Merging Dataframes y1 and y2 to get the final dataframe to feed into BI Dashbaord

In [38]:
overall = pd.concat([campaign2, non_campaign2], ignore_index=True)
overall.shape

(334, 17)

In [31]:
bajaj = pd.concat([campaign2, non_campaign2], ignore_index=True)
bajaj.shape

(334, 17)

In [39]:
keyword_count = pd.concat([overall, bajaj], ignore_index=True)
keyword_count

,Communities,Keywords,Feb-21,Mar-21,Apr-21,May-21,Jun-21,Jul-21,Aug-21,Sep-21,Oct-21,Nov-21,Dec-21,Jan-22,Feb-22,Mar-22,Apr-22
0,Overall,#,1109,2168,1630,1473,3345,3177,1976,1164,2757,3244,2120,1896,1984,1046,1015
1,Overall,Warm Oil,5,9,4,2,2,14,14,7,17,13,1,11,0,1,3
2,Overall,Avocado paste,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Overall,Coconut oil,56,50,37,44,65,124,55,22,119,37,16,55,8,20,11
4,Overall,Yogurt,22,9,21,5,27,90,44,15,125,82,8,33,8,8,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
663,Bajaj-specific,Wella,0,3,3,14,13,11,6,6,3,2,11,27,5,2,1
664,Bajaj-specific,Purplle,11,17,7,10,14,9,2,13,10,10,4,5,8,4,3
665,Bajaj-specific,Amazon,63,46,24,42,52,39,20,44,28,73,42,41,22,15,21
666,Bajaj-specific,Flipkart,3,9,7,22,10,5,7,8,10,11,6,6,6,5,2


In [40]:
keyword_count.to_csv('bajaj-keyword_mentions.csv',index=False)
# files.download('keyword_mentions.xlsx')

### Brand-wise Consideration Factors

In [171]:
y = pd.read_pickle('bajaj-convdump1_0.pickle')
y.shape

(452885, 174)

In [55]:
print(y.shape)
y.createdatutc.head()

(247803, 206)


0   2021-02-01
1   2021-02-01
2   2021-02-01
3   2021-02-01
4   2021-02-01
Name: createdatutc, dtype: datetime64[ns]

In [173]:
# y = y[y.groupClass == 'Bajaj']
y

,groupClass,groupid,createdatutc,groupname,textlower,senti,type,#,Warm Oil,Avocado paste,...,Dabur Vatika,Emami,Herbal essence,Himalaya,Khadi,Wella,Purplle,Amazon,Flipkart,Nykaa
0,Bajaj,9836d720-5665-46d4-bebe-ae8866bc6307,2021-04-01,health and beauty tips,"premium alovera gel jar \niso, gmp, natural ce...",positive,Post,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Bajaj,9836d720-5665-46d4-bebe-ae8866bc6307,2021-02-01,health and beauty tips,some homemade remedies for your hair and skin,neutral,Post,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Bajaj,9836d720-5665-46d4-bebe-ae8866bc6307,2021-03-01,health and beauty tips,you can add these ingredients into your shampoo,neutral,Post,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Bajaj,9836d720-5665-46d4-bebe-ae8866bc6307,2021-03-01,health and beauty tips,some simple hair care tips,positive,Post,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Bajaj,9836d720-5665-46d4-bebe-ae8866bc6307,2021-03-01,health and beauty tips,some home remedies for dandruff !,negative,Post,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
452880,Non-Bajaj,7920405f-a8e4-4a4d-b69f-c1fc85b4d601,2022-03-01,Arpita's & Her Team সাজকথা( female group),soma ghosh 280:/\n6pc,neutral,Comment,0,0,0,...,0,0,0,0,0,0,0,0,0,0
452881,Non-Bajaj,7920405f-a8e4-4a4d-b69f-c1fc85b4d601,2022-04-01,Arpita's & Her Team সাজকথা( female group),pp,neutral,Comment,0,0,0,...,0,0,0,0,0,0,0,0,0,0
452882,Non-Bajaj,7920405f-a8e4-4a4d-b69f-c1fc85b4d601,2022-04-01,Arpita's & Her Team সাজকথা( female group),offferrpp,neutral,Comment,0,0,0,...,0,0,0,0,0,0,0,0,0,0
452883,Non-Bajaj,7920405f-a8e4-4a4d-b69f-c1fc85b4d601,2022-04-01,Arpita's & Her Team সাজকথা( female group),pp,neutral,Comment,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [56]:
brands = list(model[model.Category == 'Brands'].Keywords)
print(f'Total Brands: {len(brands)}')
print(f'Brands in model: {brands}')

print()

cf_main = list(model[(model.Category == 'Consideration Factors') & (model.Superset != 'Appliances')].Keywords)
print(f'Main Consideration Factors: {len(cf_main)}')
print(f'Main Consideration Factors in model: {cf_main}')

print()

cf_app = list(model[(model.Category == 'Consideration Factors') & (model.Superset == 'Appliances')].Keywords)
print(f'Main Consideration Factors: {len(cf_app)}')
print(f'Main Consideration Factors in model: {cf_app}')

print()
(len(cf_main) +len(cf_app)) * len(brands)

Total Brands: 106
Brands in model: ['Dyson', 'Philips', 'Ikonic', 'Vega', 'Nova', 'Syska', 'Remington', 'Havell', 'Bblunt', 'Godrej', 'Wella Hair Color', 'Bigen', 'Vegetal', 'Streax Color', 'Cuticolor', 'Naturigin', 'Arctic Fox', 'Pravana', 'Clairol', 'Schwarzkopf', 'Loreal Hair Color', 'Revlon', 'Indus Valley', 'Vcare', 'Biotique Hair Color', 'Khadi Hair color', 'Garnier Hair color', 'Keya seth', 'Matrix', 'Bio organic', 'Bajaj Oil', 'Natvy Soul Oil', 'Kesh King Oil', 'Parachute', 'Bodywise', 'Sesa', 'Himalaya Oil', 'Navratna', 'Beardo Oil', 'Emami Hair Oil', 'Olixo', 'Sotrue', 'Rey Naturals', 'Jasmine Oil', 'Indulekha Oil', 'Dabur Amla', 'Mamaearth Oil', 'Patanjali Oil', 'Biotique Oil', 'Wow Skin Science Oil', 'Plum Oil', 'Bajaj Serum', 'Natvy Soul Serum', 'Livon', 'Streax Serum', 'Beardo Serum', 'Biolage Serum', 'Loreal Serum', 'Matrix Serum', 'Biolage Shampoo/Conditioner', 'Wow Serum', 'Biotique Serum', 'Mamaearth Serum', 'St Botanica Serum', 'Bblunt Serum', 'Khadi Serum', 'Plum Se

2544

In [57]:
print('Keywords in model but not in data:')
print(set(model.Keywords) - set(y.columns[6:]))

print('Keywords in data but not in model:')
print(set(y.columns[7:]) - set(model.Keywords))

Keywords in model but not in data:
set()
Keywords in data but not in model:
set()


In [63]:
date_with_cf_main = cf_main.copy()
date_with_cf_main.insert(0, 'createdatutc')
# date_with_cf_main.insert(1, 'groupClass')
print(date_with_cf_main)

print()

date_with_cf_app = cf_app.copy()
date_with_cf_app.insert(0, 'createdatutc')
# date_with_cf_app.insert(1, 'groupClass')
print(date_with_cf_app)

['createdatutc', 'Styling', 'Natural', 'Price', 'Paraffin free', 'Fragrance', 'Non Sticky', 'Chemical free', 'Nourishment', 'Moisturize', 'Strengthen', 'Hair Growth', 'Cures Dandruff', 'Stops Hairfall', 'Soft & Smooth Hair', 'Detangle', 'Stops Itch', 'Treats Lice', 'Grey hair treatment']

['createdatutc', 'Corded/Cordless', 'Battery Backup', 'Battery Charge Time', 'Price Appliances', 'Service', 'Warranty']


In [156]:
def getBrandAttributes(df, brand, dt_columns=False):
    
    if dt_columns == True:
        df = df.reset_index()
        df.createdatutc = pd.to_datetime(df.createdatutc)
        df = df.set_index('createdatutc').sort_index().T
        df.columns = df.columns.strftime('%b-%y')
        df.reset_index(inplace=True)
        df.rename_axis(None,axis=1,inplace=True)
        df.rename(columns={df.columns[0]:'Consideration Factors'}, inplace=True)
        col = list(df.columns)
        col.insert(0, 'Brand')
        df['Brand'] = brand
        df = df.loc[:, col]
        return df
    else:
        df = df.stack().reset_index()
        df.columns = ['createdatutc', 'Consideration Factors', 'Count']
        col = list(df.columns)
        col.insert(1, 'Brand')
        df['Brand'] = brand
        df = df.loc[:, col]
        return df

In [59]:
cf_df_main = pd.DataFrame(cf_main, columns=['Consideration Factors'])
cf_df_app = pd.DataFrame(cf_app, columns=['Consideration Factors'])

print(cf_df_main)
print(cf_df_app)

   Consideration Factors
0                Styling
1                Natural
2                  Price
3          Paraffin free
4              Fragrance
5             Non Sticky
6          Chemical free
7            Nourishment
8             Moisturize
9             Strengthen
10           Hair Growth
11        Cures Dandruff
12        Stops Hairfall
13    Soft & Smooth Hair
14              Detangle
15            Stops Itch
16           Treats Lice
17   Grey hair treatment
  Consideration Factors
0       Corded/Cordless
1        Battery Backup
2   Battery Charge Time
3      Price Appliances
4               Service
5              Warranty


In [60]:
# brand = 'Mamaearth'

# if model[model.Keywords == brand].Superset.values[0] == 'Appliances':
#     date_with_cf = date_with_cf_app
#     cf_df = cf_df_app
# else:
#     date_with_cf = date_with_cf_main
#     cf_df = cf_df_main
# brand_df = y[y[brand] == 1].loc[:, date_with_cf].reset_index(drop=True)
# # brand_df.createdatutc = pd.to_datetime(brand_df.createdatutc).dt.strftime('%B-%Y')
# grp_df = brand_df.groupby(['createdatutc']).sum().T
# # grp_df = grp_df.stack().reset_index().set_index(['createdatutc','groupClass','level_2']).unstack(level=0).droplevel(0, axis=1)
# grp_df.columns = grp_df.columns.strftime('%B-%Y')
# grp_df.reset_index(inplace=True)
# new_col = []
# for col in grp_df.columns[2:]:
#     col = pd.to_datetime(col)
#     new_col.append(col)
# grp_df.columns[2:] = new_col
# grp_df.rename(columns={'index':'Consideration Factors'}, inplace=True)
# grp_df.rename_axis(None, axis=1, inplace=True)
# grp_df = grp_df.merge(cf_df, 'right', on='Consideration Factors')
# grp_df.insert(0, 'Brand', brand)
# grp_df

In [61]:
# brands_cf = []
# total_time = 0

# for brand in brands:
    
#     # print(f'getting consideration factors for {brand}...')
#     tick = time.time()
#     brand_df = y[y[brand] == 1].loc[:, date_with_cf].reset_index(drop=True)
#     # brand_df.createdatutc = pd.to_datetime(brand_df.createdatutc).dt.strftime('%B-%Y')
#     grp_df = brand_df.groupby(['createdatutc']).sum()
#     brands_cf.append(getBrandAttributes(grp_df, brand, dt_columns=True))
#     tock = time.time()
#     exec_time = round(tock-tick, 2)
#     total_time += exec_time
#     # print(f'completed for {brand} in {exec_time} seconds\n')

# print(f'Total runtime: {round(total_time, 2)} seconds\n') 

# brxcf = pd.concat(brands_cf, ignore_index=True)
# brxcf.fillna(0, inplace=True)
# brxcf[brxcf.columns[2:]] = brxcf[brxcf.columns[2:]].astype('int32')
# brxcf.shape

In [64]:
brands_cf = []
total_time = 0

for brand in brands:
    
    # print(f'getting consideration factors for {brand}...')
    tick = time.time()
    if model[model.Keywords == brand].Superset.values[0] == 'Appliances':
        date_with_cf = date_with_cf_app
        cf_df = cf_df_app
    else:
        date_with_cf = date_with_cf_main
        cf_df = cf_df_main
    brand_df = y[y[brand] == 1].loc[:, date_with_cf].reset_index(drop=True)
    
    # brand_df.createdatutc = pd.to_datetime(brand_df.createdatutc).dt.strftime('%B-%Y')
    grp_df = brand_df.groupby(['createdatutc']).sum().T
    # grp_df = grp_df.stack().reset_index().set_index(['createdatutc','groupClass','level_2']).unstack(level=0).droplevel(0, axis=1)
    grp_df.columns = grp_df.columns.strftime('%B-%Y')
    grp_df.reset_index(inplace=True)
    for col in grp_df.columns[2:]:
        col = pd.to_datetime(col)
    grp_df.rename(columns={'index':'Consideration Factors'}, inplace=True)
    grp_df.rename_axis(None, axis=1, inplace=True)
    grp_df = grp_df.merge(cf_df, 'right', on='Consideration Factors')
    grp_df.insert(0, 'Brand', brand)
    # print(brand, grp_df.shape)
    brands_cf.append(grp_df)
    tock = time.time()
    exec_time = round(tock-tick, 2)
    total_time += exec_time
    # print(f'completed for {brand} in {exec_time} seconds\n')

print(f'Total runtime: {round(total_time, 2)} seconds\n') 

brxcf = pd.concat(brands_cf, ignore_index=True)
brxcf.fillna(0, inplace=True)
brxcf[brxcf.columns[2:]] = brxcf[brxcf.columns[2:]].astype('int32')
date_df = brxcf[brxcf.columns[2:]]
date_df.columns = pd.to_datetime(date_df.columns)
date_df = date_df[sorted(date_df.columns)]
date_df.columns = date_df.columns.strftime('%b-%y')
brxcf = pd.concat([brxcf.iloc[:,:2], date_df], axis=1)
brxcf.shape

Total runtime: 1.77 seconds



(1812, 17)

In [65]:
brxcf.head()

,Brand,Consideration Factors,Feb-21,Mar-21,Apr-21,May-21,Jun-21,Jul-21,Aug-21,Sep-21,Oct-21,Nov-21,Dec-21,Jan-22,Feb-22,Mar-22,Apr-22
0,Dyson,Corded/Cordless,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Dyson,Battery Backup,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Dyson,Battery Charge Time,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Dyson,Price Appliances,3,1,3,1,0,1,0,0,0,0,0,0,0,0,0
4,Dyson,Service,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [90]:
brxcf.to_csv('battri_test.csv', index=False)

In [176]:
overall_brxcf = brxcf.copy()
overall_brxcf.insert(0, 'Communities', 'Overall')
overall_brxcf.head()

,Communities,Brand,Consideration Factors,Feb-21,Mar-21,Apr-21,May-21,Jun-21,Jul-21,Aug-21,Sep-21,Oct-21,Nov-21,Dec-21,Jan-22,Feb-22,Mar-22,Apr-22
0,Overall,Dyson,Corded/Cordless,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Overall,Dyson,Battery Backup,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Overall,Dyson,Battery Charge Time,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Overall,Dyson,Price Appliances,3,1,3,1,0,1,0,0,0,0,0,0,0,0,0
4,Overall,Dyson,Service,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [170]:
bajaj_brxcf = brxcf.copy()
bajaj_brxcf.insert(0, 'Communities', 'Bajaj')
bajaj_brxcf.head()

,Communities,Brand,Consideration Factors,Feb-21,Mar-21,Apr-21,May-21,Jun-21,Jul-21,Aug-21,Sep-21,Oct-21,Nov-21,Dec-21,Jan-22,Feb-22,Mar-22,Apr-22
0,Bajaj,Dyson,Corded/Cordless,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Bajaj,Dyson,Battery Backup,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Bajaj,Dyson,Battery Charge Time,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Bajaj,Dyson,Price Appliances,3,1,3,1,0,1,0,0,0,0,0,0,0,0,0
4,Bajaj,Dyson,Service,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [177]:
brxcf0 = pd.concat([overall_brxcf, bajaj_brxcf], ignore_index=True)

In [13]:
brxcf0 = pd.read_csv('bajaj-brandattributes.csv')
brxcf0.head()

,Communities,Brand,Consideration Factors,Feb-21,Mar-21,Apr-21,May-21,Jun-21,Jul-21,Aug-21,Sep-21,Nov-21,Dec-21,Jan-22,Feb-22,Mar-22,Apr-22,Oct-21
0,Overall,Dyson,Styling,34,18,29,4,6,39,91,4,0,0,0,0,0,0,0
1,Overall,Dyson,Moisturize,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Overall,Dyson,Natural,0,2,4,2,2,7,1,2,0,0,0,0,0,0,0
3,Overall,Dyson,Strengthen,1,1,0,0,0,5,0,0,0,0,0,0,0,0,0
4,Overall,Dyson,Price,3,1,3,1,0,1,0,0,0,0,0,0,0,0,0


In [72]:
brand_attributes = model[['Keywords','Superset']].merge(brxcf, 'right', left_on='Keywords', right_on='Brand').drop('Keywords', axis=1)

In [73]:
brand_attributes = model[['Keywords','Subset']].merge(brand_attributes, 'right', right_on='Brand', left_on='Keywords')

In [75]:
brand_attributes.drop(columns=['Keywords','Brand'], inplace=True)

In [76]:
brand_attributes

,Subset,Superset,Consideration Factors,Feb-21,Mar-21,Apr-21,May-21,Jun-21,Jul-21,Aug-21,Sep-21,Oct-21,Nov-21,Dec-21,Jan-22,Feb-22,Mar-22,Apr-22
0,Dyson,Appliances,Corded/Cordless,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Dyson,Appliances,Battery Backup,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Dyson,Appliances,Battery Charge Time,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Dyson,Appliances,Price Appliances,3,1,3,1,0,1,0,0,0,0,0,0,0,0,0
4,Dyson,Appliances,Service,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1807,Nykaa,Shopping Platform,Soft & Smooth Hair,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0
1808,Nykaa,Shopping Platform,Detangle,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1809,Nykaa,Shopping Platform,Stops Itch,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1810,Nykaa,Shopping Platform,Treats Lice,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [77]:
brand_attributes.to_csv('bajaj-brandattributes_new.csv', index=False)